# Main Program Testing 
Testing the entire program as a singular file.
However, most of the displying will be done in other notebooks.


In [1]:
# Imports
import torch
from torch import nn
import matplotlib.pyplot as plt
import rasterio
import spectral
import os
import numpy as np
from spectral import open_image

In [2]:
# Path to dataset (input folder)
dirpath = "/home/troy/sweet_potato_data_combined"

# Check what is inside your input folder 
def check_folder(dirpath):
    for dirpath, dirnames, filenames in os.walk(dirpath):
        print(f"There are {len(dirnames)} directories (internal folders) and {len(filenames)} images in the path at'{dirpath}'")
        
check_folder(dirpath)

There are 0 directories (internal folders) and 1912 images in the path at'/home/troy/sweet_potato_data_combined'


Split the data from the dataset folder by file type

In [3]:
# Function to split files into inside input folder by respective file extention
# Input: Dataset folder
# Return: 3 lists
def split_files(dirpath):
    # Get a list of all files in the folder
    files = os.listdir(dirpath)
    
    # Sort files numerically begining at 1
    files = sorted(files, key=lambda x: int(''.join(filter(str.isdigit, x)) or -1))
    
    # Filter out .hdr files, data files, and .png files into their own list
    hdr_files = [f for f in files if f.endswith('.hdr')]
    data_files = [f for f in files if not f.endswith('.hdr') and not f.endswith('.png')]
    mask_files = [f for f in files if f.endswith('.png')]
    
    # Return Each file type as a list
    return hdr_files, data_files, mask_files
# list_envi_files(dirpath)
hdr_files, data_files, mask_files = split_files(dirpath)

In [4]:
print(hdr_files)
print(data_files)
print(mask_files)

['sweet_potato_leaf_1.hdr', 'sweet_potato_leaf_2.hdr', 'sweet_potato_leaf_3.hdr', 'sweet_potato_leaf_4.hdr', 'sweet_potato_leaf_5.hdr', 'sweet_potato_leaf_6.hdr', 'sweet_potato_leaf_7.hdr', 'sweet_potato_leaf_8.hdr', 'sweet_potato_leaf_9.hdr', 'sweet_potato_leaf_10.hdr', 'sweet_potato_leaf_11.hdr', 'sweet_potato_leaf_12.hdr', 'sweet_potato_leaf_13.hdr', 'sweet_potato_leaf_14.hdr', 'sweet_potato_leaf_15.hdr', 'sweet_potato_leaf_16.hdr', 'sweet_potato_leaf_17.hdr', 'sweet_potato_leaf_18.hdr', 'sweet_potato_leaf_19.hdr', 'sweet_potato_leaf_20.hdr', 'sweet_potato_leaf_21.hdr', 'sweet_potato_leaf_22.hdr', 'sweet_potato_leaf_23.hdr', 'sweet_potato_leaf_24.hdr', 'sweet_potato_leaf_25.hdr', 'sweet_potato_leaf_26.hdr', 'sweet_potato_leaf_27.hdr', 'sweet_potato_leaf_28.hdr', 'sweet_potato_leaf_29.hdr', 'sweet_potato_leaf_30.hdr', 'sweet_potato_leaf_31.hdr', 'sweet_potato_leaf_32.hdr', 'sweet_potato_leaf_33.hdr', 'sweet_potato_leaf_34.hdr', 'sweet_potato_leaf_35.hdr', 'sweet_potato_leaf_36.hdr', 

Group 3 types into its own set.
A single Image contains 3 files:
1 x .HDR file
1 x data file (no extension)
1 x Masked image .png file

In [6]:
def group_files(dirpath):
    hdr_files, data_files, mask_files = split_files(dirpath)
    
    paired_files = []
    for hdr_file in hdr_files:
        base_name = hdr_file[:-4]
        data_file = next((f for f in data_files if f.startswith(base_name)), None)
        mask_file = next((f for f in mask_files if f.startswith(base_name)), None)
        if data_file and mask_file:
            paired_files.append((os.path.join(dirpath, hdr_file),
                                 os.path.join(dirpath, data_file),
                                 os.path.join(dirpath, mask_file)))
    # Return the 3 types of files as a list
    return paired_files

Check the paired files by printing out
Yes paired means 2 but in this case we have 3 files.

In [13]:
# Assuming images is a list of grouped image sets
images = group_files(dirpath)

# Variable to control the number of iterations
num = 10

# Iterate over the images with an index
for index, image_set in enumerate(images):
    # Print the current image set
    print(image_set)
    
    # Stop the loop if the desired number of iterations is reached
    if index + 1 == num:
        break
print("\nThe number of files paired (in a set of 3) is: ", len(images))

('/home/troy/sweet_potato_data_combined/sweet_potato_leaf_1.hdr', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_1', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_1_gt.png')
('/home/troy/sweet_potato_data_combined/sweet_potato_leaf_2.hdr', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_2', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_2_gt.png')
('/home/troy/sweet_potato_data_combined/sweet_potato_leaf_3.hdr', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_3', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_3_gt.png')
('/home/troy/sweet_potato_data_combined/sweet_potato_leaf_4.hdr', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_4', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_4_gt.png')
('/home/troy/sweet_potato_data_combined/sweet_potato_leaf_5.hdr', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_5', '/home/troy/sweet_potato_data_combined/sweet_potato_leaf_5_gt.png')
('/home/troy/sweet_potato